# 🐾 ZOO Classification — Notebook Corregido

## 📥 Cargar dataset

In [8]:

import pandas as pd

zoo = pd.read_csv('CSV/zoo.data')
zoo.head()


,aardvark,1,0,0.1,1.1,0.2,0.3,1.2,1.3,1.4,1.5,0.4,0.5,4,0.6,0.7,1.6,1.7
0,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
1,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
2,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
3,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
4,buffalo,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1


In [9]:
with open("CSV/zoo.names") as f:
    contenido = f.read()

print(contenido)


1. Title: Zoo database

2. Source Information
   -- Creator: Richard Forsyth
   -- Donor: Richard S. Forsyth 
             8 Grosvenor Avenue
             Mapperley Park
             Nottingham NG3 5DX
             0602-621676
   -- Date: 5/15/1990
 
3. Past Usage:
   -- None known other than what is shown in Forsyth's PC/BEAGLE User's Guide.

4. Relevant Information:
   -- A simple database containing 17 Boolean-valued attributes.  The "type"
      attribute appears to be the class attribute.  Here is a breakdown of
      which animals are in which type: (I find it unusual that there are
      2 instances of "frog" and one of "girl"!)

      Class# Set of animals:
      ====== ===============================================================
           1 (41) aardvark, antelope, bear, boar, buffalo, calf,
                  cavy, cheetah, deer, dolphin, elephant,
                  fruitbat, giraffe, girl, goat, gorilla, hamster,
                  hare, leopard, lion, lynx, mink, mole, mo

In [10]:
# Nombres de columnas oficiales del dataset
col_names = [
    "animal_name","hair","feathers","eggs","milk","airborne","aquatic","predator",
    "toothed","backbone","breathes","venomous","fins","legs","tail",
    "domestic","catsize","type"
]

# Cargar el archivo
data_path = "/mnt/data/zoo.data"
zoo = pd.read_csv("CSV/zoo.data", header=None, names=col_names)


# Columnas binarias (todas excepto animal_name y legs)
binary_cols = zoo.columns.difference(["animal_name", "legs", "type"])


# Diccionario para decodificar el tipo de animal
type_map = {
    1: "Mammal",
    2: "Bird",
    3: "Reptile",
    4: "Fish",
    5: "Amphibian",
    6: "Bug",
    7: "Invertebrate"
}

zoo["type"] = zoo["type"].replace(type_map)

# Mostrar primeras filas ya legibles
zoo.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,Mammal
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,Mammal
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,Fish
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,Mammal
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,Mammal


In [11]:
zoo.columns

Index(['animal_name', 'hair', 'feathers', 'eggs', 'milk', 'airborne',
       'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous',
       'fins', 'legs', 'tail', 'domestic', 'catsize', 'type'],
      dtype='object')

## 🧹 Preprocesado

In [12]:
# Eliminamos nombre y definimos target
X = zoo.drop(columns=['animal_name','type'])
y = zoo['type']

X.shape, y.shape


((101, 16), (101,))

## ✂️ Train / Test Split

In [13]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train.shape, X_test.shape


((70, 16), (31, 16))

## 📏 Escalado

In [14]:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## 🤖 Entrenar modelos

In [15]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

models = {
    'LogReg': LogisticRegression(max_iter=500),
    'Tree': DecisionTreeClassifier(),
    'RF': RandomForestClassifier(),
    'GB': GradientBoostingClassifier()
}

results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    pred = model.predict(X_test_scaled)
    results[name] = accuracy_score(y_test, pred)

results


{'LogReg': 0.967741935483871,
 'Tree': 0.967741935483871,
 'RF': 0.967741935483871,
 'GB': 0.9032258064516129}

## ⭐ Selección del mejor modelo

In [16]:

best_model_name = max(results, key=results.get)
best_model = models[best_model_name]
best_model_name


'LogReg'

## 🐶 Predicción de un nuevo animal

In [17]:

# Crear mamífero de ejemplo con el ORDEN EXACTO DE X
nuevo = pd.DataFrame([{
    'hair':1, 'feathers':0, 'eggs':0, 'milk':1, 'airborne':0, 'aquatic':0,
    'predator':0, 'toothed':1, 'backbone':1, 'breathes':1, 'venomous':0,
    'fins':0, 'legs':4, 'tail':1, 'domestic':0, 'catsize':1
}])

nuevo = nuevo[X.columns]
nuevo_scaled = scaler.transform(nuevo)

pred = best_model.predict(nuevo_scaled)
pred[0]


'Mammal'

In [18]:
nuevo_prueba = pd.DataFrame([{
    'hair':0,
    'feathers':1,
    'eggs':1,
    'milk':0,
    'airborne':1,
    'aquatic':0,
    'predator':0,
    'toothed':0,
    'backbone':1,
    'breathes':1,
    'venomous':0,
    'fins':0,
    'legs':2,
    'tail':1,
    'domestic':0,
    'catsize':0
}])

# Respetar el orden exacto de X
nuevo_prueba = nuevo_prueba[X.columns]

# Escalado
nuevo_prueba_scaled = scaler.transform(nuevo_prueba)

# Predicción
pred_prueba = best_model.predict(nuevo_prueba_scaled)
pred_prueba[0]

'Bird'

In [19]:
nuevo_prueba = pd.DataFrame([{
    'hair':1,
    'feathers':0,
    'eggs':0,
    'milk':1,
    'airborne':1,
    'aquatic':0,
    'predator':0,
    'toothed':1,
    'backbone':1,
    'breathes':1,
    'venomous':0,
    'fins':1,
    'legs':4,
    'tail':0,
    'domestic':0,
    'catsize':0
}])

# Respetar el orden exacto de X
nuevo_prueba = nuevo_prueba[X.columns]

# Escalado
nuevo_prueba_scaled = scaler.transform(nuevo_prueba)

# Predicción
pred_prueba = best_model.predict(nuevo_prueba_scaled)
pred_prueba[0]

'Mammal'

Creación de Streamlit

In [21]:
import os
import joblib

os.makedirs("models", exist_ok=True)

joblib.dump(best_model, "models/best_model.pkl")
joblib.dump(scaler, "models/scaler.pkl")
joblib.dump(X.columns, "models/columns.pkl")


['models/columns.pkl']

In [22]:
import os
os.listdir("models")


['best_model.pkl', 'columns.pkl', 'scaler.pkl']